# Speed Tests

In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib inline
%matplotlib widget

In [2]:
import numpy as np
from nn import Network
from nnbench import NetMaker, NNMEG

# Is `NNMEG` slower than `Network`
Does `NNMEG`, the instrumented subclass of `Network`, cost us much in speed?

In [3]:
nm = NetMaker()
net = nm('1x8tx8tx3tx3t')

In [4]:
mnm = NetMaker(NNMEG)
mnet = mnm('1x8tx8tx3tx3t')
starting_sv = net.state_vector()
mnet.set_state_from_vector(starting_sv)

## 3-bit linear binary output

In [5]:
def adc(input):
    m = max(0, min(7, int(8*input)))
    return np.array([(m>>2)&1, (m>>1)&1, m&1]) * 2 - 1

vadc = lambda v: np.array([adc(p) for p in v])

## Training data

In [6]:
x = np.arange(0, 1, 1.0/(8*8)).reshape(-1,1) # 8 points in each output region
training_batch_cluster = [(x, vadc(x))]

In [7]:
training_batch_clusters = [[(x, vadc(x))] for x in (np.arange(0, 1, 1.0/(8*(1<<i))).reshape(-1,1) for i in range(10))]

In [8]:
training_batch_clusters[0]

[(array([[0.   ],
         [0.125],
         [0.25 ],
         [0.375],
         [0.5  ],
         [0.625],
         [0.75 ],
         [0.875]]),
  array([[-1, -1, -1],
         [-1, -1,  1],
         [-1,  1, -1],
         [-1,  1,  1],
         [ 1, -1, -1],
         [ 1, -1,  1],
         [ 1,  1, -1],
         [ 1,  1,  1]]))]

In [9]:
len(training_batch_clusters[-1][-1][0])

4096

In [10]:
np.all(net(x) == mnet(x))

True

## A short batch

In [11]:
net.set_state_from_vector(starting_sv)
mnet.set_state_from_vector(starting_sv)

In [12]:
%timeit net.learn(training_batch_clusters[0])

579 µs ± 5.17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%timeit mnet.learn(training_batch_clusters[0])

592 µs ± 3.37 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
%timeit mnet.deltas()

27.3 µs ± 428 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## A long batch

In [15]:
net.set_state_from_vector(starting_sv)
mnet.set_state_from_vector(starting_sv)

In [16]:
%timeit net.learn(training_batch_clusters[9])

216 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%timeit mnet.learn(training_batch_clusters[9])

218 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit mnet.deltas()

26.3 µs ± 548 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Answers:
* It adds < 3% delay if we don't call `deltas`
* `deltas()` is fast and independent of batch size